<a target="_blank" href="https://colab.research.google.com/github/tatsath/Interpretability/blob/main/ActivationDetectTwoPass.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [161]:
!pip install nnsight sae-lens

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [162]:
!pip install --upgrade --force-reinstall numpy scipy scikit-learn

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.6.0
    Uninstalling threadpoolctl-3.6.0:
      Successfully uni

In [163]:
!pip install neuronpedia

In [164]:
#!pip install --upgrade --force-reinstall numpy

In [1]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

import huggingface_hub


# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:0", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 6

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:0'
)
# Load the SAE weights using safetensors
# from safetensors import safe_open
# with safe_open("/home/ashater/work/sae.safetensors", framework="pt") as st:
#     w_dec = st.get_tensor("W_dec").to('cuda:1')





In [2]:

from huggingface_hub import hf_hub_download

# Define the repository ID and filename
repo_id = "jbloom/Gemma-2b-Residual-Stream-SAEs"
filename = "sae_weights.safetensors"

# Download the file


# filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.6.hook_resid_post_16384")

#Layer 12
#filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.12.hook_resid_post_16384")

# Layer 6
filepath = hf_hub_download(repo_id=repo_id, filename=filename, subfolder="gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr")

# Print the downloaded file path
print(filepath)

# Print the downloaded file path
print(filepath)

/root/.cache/huggingface/hub/models--jbloom--Gemma-2b-Residual-Stream-SAEs/snapshots/2e64e9127f413318e0500a6ba3981483f251bcee/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors
/root/.cache/huggingface/hub/models--jbloom--Gemma-2b-Residual-Stream-SAEs/snapshots/2e64e9127f413318e0500a6ba3981483f251bcee/gemma_2b_blocks.6.hook_resid_post_16384_anthropic_fast_lr/sae_weights.safetensors


In [3]:
# Load the SAE weights using safetensors
from safetensors import safe_open
with safe_open(filepath, framework="pt") as st:
    w_dec = st.get_tensor("W_dec").to('cuda:0')

In [4]:
import torch
#torch.cuda.is_available()

#torch.cuda.device_count()
#len(docs)
torch.cuda.set_device(0)
torch.cuda.current_device()


0

In [5]:
#sae.W_dec == w_dec

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(w_dec[471,:], sae.W_dec).topk(100).indices

tensor([  471,  4579, 11129, 11728,  6446,  6510, 14611,  7687,  5411,   891,
        15780,  7372,   675, 13371,   713, 15976,  1076,  5419,   976, 13837,
        10036,  2817,  6495,   154,  2287, 15131, 16291,  3466,  2111, 14557,
         9715,  8481,  8099,  8567, 15023, 11584,  3869,  7062, 15369,  3864,
        14291,  5239,  7807,  4189,  4815, 13685, 10031,  2126,  5936, 14224,
        15698, 14444, 10105, 10185,  5407,  6545,  8586,  8753,  6027, 13939,
        11269,  9963,  8461, 15760,  2312,   544,  9188, 11808,  8498,   354,
         4989,  7682, 11355,  4828, 14022,  3586,  8306,   954, 12170,   167,
        11255,  9691,  6426, 11718, 14749,  7608, 14153,  3971, 14416, 13584,
         3900, 13970,  8222, 14874,  8523,  9004, 12520, 15384,  1805,   575],
       device='cuda:0')

Parameters

In [6]:
#@title Self-explanation in 18 lines
feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
# feature = 3629
scale = 1 #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 200  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
# vals = ['positive', 'neutral', 'negative']
# samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

## Main code prompt


In [30]:
full_res = []

prompt = "Credit Risk related"
# prompt = """Credit Risk Related"""




with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[6].output.save()

features = sae.encode(resid[0])

summed_activations = features.abs().sum(dim=1) # Sort by max activations

print(summed_activations)
summed_activations = features
top_activations_indices = summed_activations.topk(1000) # Get indices of top 20
print(top_activations_indices)
#top_activations_indices = summed_activations.indices # Get indices of top 20


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    #print(model.tokenizer.batch_decode(out))
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    #print(prompt + "|"+ s + "|" + str(top_activations_indices))
    #print(prompt + "|"+ l +"\n" + "|" + str(top_activations_indices))
    #print(str(top_activations_indices))


tensor([[ 2.6458, 19.0831,  0.0000,  ...,  0.0000, 11.5059,  0.0000]],
       device='cuda:0', grad_fn=<SumBackward1>)
torch.return_types.topk(
values=tensor([[[51.1168, 48.4639, 45.8360,  ..., 18.9703, 18.9648, 18.9466],
         [13.9223,  3.4922,  2.9306,  ...,  0.0000,  0.0000,  0.0000],
         [12.2706,  5.2049,  2.2141,  ...,  0.0000,  0.0000,  0.0000],
         [11.3995,  4.3096,  2.9391,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([[[ 3390, 15881,  5347,  ...,  8207,    15,  3380],
         [ 5419, 10035,   471,  ...,   769,   770,   771],
         [10870,   471, 12704,  ...,   899,   900,   901],
         [15570,  4633, 11912,  ...,   889,   890,   891]]], device='cuda:0'))


https://www.neuronpedia.org/gemma-2b/6-res-jb/471


## summed activations across tokens, finance features

In [8]:
s

'" to Unqualified Borrowers\\n\\nAs a lender, I need to identify and manage credit risk related to borrowers who are not qualified for a loan. Identifying such borrowers can be challenging, as it requires careful assessment of various factors, such as financial information, credit history, and collateral pledged.\\n\\n**Challenges in Identifying Unqualified Borrowers:**\\n\\n* **Complex Due Diligence:** Assessing the financial viability of borrowers and determining their ability to repay loans can be complex due to factors like self-employment, limited financial records, and complex credit lines with multiple issuers.\\n* **Lack of Historical Data:** Many borrowers do not have complete or reliable credit history, making it difficult to identify early signs of financial trouble.\\n* **Collateral Limitations:** Some borrowers may not have sufficient collateral to secure a loan, even if they have good credit.\\n\\n**Credit Risk Management Strategies for Unqualified Borrowers:**\\n\\n* **T

In [9]:
 #model.tokenizer.decode(model.tokenizer.encode(prompt)[2])
summed_activations[0][0]


tensor([ 2.6458, 19.0831,  0.0000,  ...,  0.0000, 11.5059,  0.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [10]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[True, False, False, False, False, False, True, False, False, False, False]

In [11]:
#summed_activations[summed_activations > 10].shape
idc = summed_activations.topk(1139).indices
[g in idc for g in [10870]]

[True]

## Get features for each token

In [12]:
idc = summed_activations.mean(dim=1).topk(5000).indices
[g in idc for g in [354, 2105, 4218, 5239, 6545, 7529, 8461, 11614, 11916, 14291, 15399]]

[False, False, False, False, True, False, False, True, False, False, True]

In [13]:
prompt = "Credit Risk"
# prompt = """Credit Risk Related"""



#prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

# prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
# positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
# print(positions)
#with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=False, scan=False, validate=False, temperature=0.0) as gen:
with model.generate(s, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
    # vector steering section
    # for feature_i in topics:
    # vector = sae.W_dec[[feature]]
    # vector = vector / vector.norm()
    # vector = vector * scale
    # setting vector into layer 2
    # for position in positions:
    #   model.model.layers[2].output[0][:, position] = vector

    # get final output saved
    out = model.generator.output.save()
    # capture at layer 6 or 12
    resid = model.model.layers[6].output.save()

features = sae.encode(resid[0])

summed_activations = features.abs().sum(dim=1) # Sort by max activations

print(summed_activations)
summed_activations = features
top_activations_indices = summed_activations.topk(1000) # Get indices of top 20
print(top_activations_indices)
#top_activations_indices = summed_activations.indices # Get indices of top 20


for i, l in enumerate(model.tokenizer.batch_decode(out)):
    #print(model.tokenizer.batch_decode(out))
    s = repr(l.partition(prompt)[2].partition("<eos>")[0])
    #print(prompt + "|"+ s + "|" + str(top_activations_indices))
    #print(prompt + "|"+ l +"\n" + "|" + str(top_activations_indices))
    #print(str(top_activations_indices))

tensor([[ 3.9316, 19.0836,  0.0000,  ...,  0.0000, 11.5040,  0.0000]],
       device='cuda:0', grad_fn=<SumBackward1>)
torch.return_types.topk(
values=tensor([[[51.1153, 48.4625, 45.8345,  ..., 18.9746, 18.9655, 18.9496],
         [ 5.7439,  3.7488,  2.1953,  ...,  0.0000,  0.0000,  0.0000],
         [ 5.1796,  3.8137,  3.1580,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 5.0569,  3.8993,  2.5219,  ...,  0.0000,  0.0000,  0.0000],
         [10.3142,  5.1849,  2.2300,  ...,  0.0000,  0.0000,  0.0000],
         [ 4.5565,  1.9661,  1.8337,  ...,  0.0000,  0.0000,  0.0000]]],
       device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([[[ 3390, 15881,  5347,  ...,  8207,    15,  3380],
         [ 2307,  2745,  3586,  ...,   943,   944,   945],
         [ 3558,  5145,  7764,  ...,   946,   947,   948],
         ...,
         [11487,  2802,  8861,  ...,   964,   965,   966],
         [ 5419,   471,  6810,  ...,   942,   943,   944],
         [ 5145,  2745,  1590,  ...,   9

In [14]:
summed_activations.mean(dim=1).topk(100)

torch.return_types.topk(
values=tensor([[0.9785, 0.7426, 0.6979, 0.5324, 0.4549, 0.4418, 0.3976, 0.3612, 0.3399,
         0.3026, 0.2807, 0.2796, 0.2417, 0.2413, 0.2394, 0.2392, 0.2347, 0.2289,
         0.2273, 0.2264, 0.2244, 0.2242, 0.2233, 0.2233, 0.2232, 0.2224, 0.2218,
         0.2193, 0.2190, 0.2166, 0.2161, 0.2131, 0.2125, 0.2125, 0.2118, 0.2072,
         0.2065, 0.2036, 0.2017, 0.1999, 0.1972, 0.1938, 0.1917, 0.1905, 0.1904,
         0.1902, 0.1897, 0.1884, 0.1882, 0.1871, 0.1864, 0.1861, 0.1852, 0.1842,
         0.1841, 0.1841, 0.1838, 0.1836, 0.1836, 0.1830, 0.1815, 0.1812, 0.1805,
         0.1796, 0.1791, 0.1784, 0.1782, 0.1775, 0.1772, 0.1758, 0.1747, 0.1739,
         0.1736, 0.1729, 0.1703, 0.1698, 0.1690, 0.1688, 0.1686, 0.1683, 0.1681,
         0.1676, 0.1674, 0.1664, 0.1663, 0.1662, 0.1657, 0.1655, 0.1655, 0.1655,
         0.1655, 0.1652, 0.1651, 0.1648, 0.1647, 0.1645, 0.1645, 0.1640, 0.1634,
         0.1628]], device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor(

In [15]:
summed_activations.sum(dim=1).topk(100)

torch.return_types.topk(
values=tensor([[211.3501, 160.3963, 150.7456, 115.0033,  98.2507,  95.4368,  85.8903,
          78.0248,  73.4147,  65.3556,  60.6345,  60.3914,  52.2032,  52.1265,
          51.7082,  51.6574,  50.6944,  49.4439,  49.0894,  48.9014,  48.4661,
          48.4284,  48.2375,  48.2284,  48.2083,  48.0293,  47.9106,  47.3755,
          47.3043,  46.7926,  46.6674,  46.0386,  45.8965,  45.8964,  45.7415,
          44.7525,  44.5941,  43.9713,  43.5592,  43.1827,  42.5932,  41.8508,
          41.4142,  41.1376,  41.1363,  41.0790,  40.9737,  40.6855,  40.6557,
          40.4169,  40.2574,  40.1891,  40.0062,  39.7880,  39.7741,  39.7681,
          39.6905,  39.6551,  39.6533,  39.5226,  39.2104,  39.1321,  38.9801,
          38.7890,  38.6802,  38.5408,  38.4970,  38.3346,  38.2650,  37.9669,
          37.7361,  37.5638,  37.4900,  37.3529,  36.7821,  36.6769,  36.4972,
          36.4612,  36.4216,  36.3522,  36.3024,  36.2101,  36.1637,  35.9337,
          35.9255,  

In [116]:
import pandas as pd
import torch

def get_top_activations(model, sae, prompt, max_new_tokens=200, n_generate=1, top_n=2):
  """
  Returns the top n activations in a Pandas DataFrame for both the prompt and the output.

  Args:
    model: The language model object.
    sae: The SAE object.
    prompt: The input prompt.
    max_new_tokens: The maximum number of new tokens to generate for the output.
    n_generate: The number of output sequences to generate.
    top_n: The number of top activations to return.

  Returns:
    A tuple containing two Pandas DataFrames: one for the prompt and one for the output.
  """

  # # Get activations for the prompt
  # with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
  #     # get final output saved
  #     out = model.generator.output.save()
  #     # capture at layer 6 or 12
  #     resid = model.model.layers[6].output.save()

  # prompt_features = sae.encode(resid[0])

  # prompt_summed_activations = features.abs().sum(dim=1) # Sort by max activations

  # print(summed_activations)
  # summed_activations = features
  # top_activations_indices = summed_activations.topk(1000) # Get indices of top 20



  with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
      # get final output saved
      out = model.generator.output.save()
      # capture at layer 6 or 12
      resid = model.model.layers[6].output.save()

  prompt_features = sae.encode(resid[0][:, 1:]) # exclude <bos> token
  #print(prompt_features.topk(top_n+1))
  prompt_summed_activations_topk  = prompt_features.abs().sum(dim=1).topk(top_n) # Sort by max activations
  prompt_mean_activations_topk  = prompt_features.abs().mean(dim=1).topk(top_n) # Sort by max activations
  #print(prompt_summed_activations_topk)
  #print(prompt_mean_activations_topk )



  for i, l in enumerate(model.tokenizer.batch_decode(out)):
      s = repr(l.partition(prompt)[2].partition("<eos>")[0])


  # Get activations for the output
  with model.generate(s, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
      # out = model.generator.output.save()
      resid = model.model.layers[6].output.save()  # Capture at layer 6

  output_features = sae.encode(resid[0][:, 1:])
  output_summed_activations_topk = output_features.abs().sum(dim=1).topk(top_n) # Sort by max activations
  output_mean_activations_topk  = output_features.abs().mean(dim=1).topk(top_n) # Sort by max activations
  #print(output_summed_activations_topk)
  #print(output_mean_activations_topk )

  # Create prompt_df
  prompt_df = pd.DataFrame({
      "feature": prompt_summed_activations_topk.indices[0].cpu().detach().numpy(),
      "sum_activation": prompt_summed_activations_topk.values[0].cpu().detach().numpy(),
      "mean_activation": prompt_mean_activations_topk.values[0].cpu().detach().numpy()
  })

  # Create output_df
  output_df = pd.DataFrame({
      "feature": output_summed_activations_topk.indices[0].cpu().detach().numpy(), # Added .detach()
      "sum_activation": output_summed_activations_topk.values[0].cpu().detach().numpy(), # Added .detach()
      "mean_activation": output_mean_activations_topk.values[0].cpu().detach().numpy() # Added .detach()
  })

  output = s

  return prompt_df, output_df, output

In [117]:
# Get the top 100 activations for both prompt and output

prompt = 'credit Risk related'
prompt_df, output_df, output = get_top_activations(model, sae, prompt, top_n=2)

# Display the DataFrames
print("Prompt activations:")
display(prompt_df)

print("\nOutput activations:")
display(output_df)

Prompt activations:


,feature,sum_activation,mean_activation
0,5419,15.582191,5.194064
1,10870,12.870153,4.290051



Output activations:


,feature,sum_activation,mean_activation
0,3553,210.532791,1.017067
1,471,136.494370,0.659393


Bias Use case

In [118]:
# Get the top 100 activations for both prompt and output

prompt = "Credit"
prompt_df, output_df, output = get_top_activations(model, sae, prompt, top_n=5)

# Display the DataFrames
print("Prompt activations:")
display(prompt_df)

print("\nOutput activations:")
display(output_df)

Prompt activations:


,feature,sum_activation,mean_activation
0,5419,13.926304,13.926304
1,10035,3.486936,3.486936
2,471,2.929296,2.929296
3,3869,2.272645,2.272645
4,9779,2.013997,2.013997



Output activations:


,feature,sum_activation,mean_activation
0,471,362.772522,1.711191
1,3464,250.467194,1.181449
2,3553,226.805420,1.069837
3,5419,166.543854,0.785584
4,6351,121.404739,0.572664


## Autointerp from Neuronpedia

In [119]:
import requests
import json
from google.colab import userdata

def get_or_generate_activation_description(modelId, layer, index, apiKey):
    """Retrieves or generates an activation description.

    Args:
        modelId: The model ID.
        layer: The layer number.
        index: The neuron index.
        apiKey: Your Neuronpedia API key.

    Returns:
        The activation description, or None if not found.
    """

    # Try to retrieve the description
    retrieve_url = f"https://neuronpedia.org/api/feature/{modelId}/{layer}/{index}"
    retrieve_response = requests.get(retrieve_url)
    retrieve_data = retrieve_response.json()
    description = find_description(retrieve_data)  # Assuming find_description is defined

    # If description is not found, generate it
    #print(description)
    if description is None:
        generate_url = "https://www.neuronpedia.org/api/explanation/generate"
        payload = {
            "modelId": modelId,
            "layer": layer,
            "index": index,
            "explanationType": "oai_attention-head",
            "explanationModelName": "gpt-4o"
        }
        headers = {
            "Content-Type": "application/json",
            "x-api-key": apiKey
        }
        generate_response = requests.post(generate_url, json=payload, headers=headers)
        generate_data = generate_response.json()
        print(generate_data)
        description = generate_data.get('explanation', {}).get('description', None)

    return description

def find_description(data):
    if 'explanations' in data:
        for item in data['explanations']:
            return item['description']
    # recursively check nested dictionaries
    for key, value in data.items():
        if isinstance(value, dict):
            result = find_description(value)
            if result:
                return result
        elif isinstance(value, list):
            for element in value:
                if isinstance(element, dict):
                    result = find_description(element)
                    if result:
                        return result
    return None


# Example usage:
apiKey = userdata.get('NEURONPEDIA_API_KEY')  # Replace with your actual API key
description = get_or_generate_activation_description("gemma-2b", "6-res-jb", "10006", apiKey)
print("Description:", description)

Description: words related to specific fruits, especially cherry


In [120]:
# ... (your previous code to define get_or_generate_activation_description and find_description) ...

# Assuming you have output_df and prompt_df DataFrames

# API key, modelId, and layer (replace with your actual values)

apiKey = userdata.get('NEURONPEDIA_API_KEY')
modelId = "gemma-2b"
layer = "6-res-jb"

# Function to get feature label (same as before)
def get_feature_label(row):
    #print(int(row['feature']))
    feature_index = int(row['feature'])
    description = get_or_generate_activation_description(modelId, layer, feature_index, apiKey)
    return description

# Apply to output_df
output_df['feature_label'] = output_df.apply(get_feature_label, axis=1)

# Apply to prompt_df
prompt_df['feature_label'] = prompt_df.apply(get_feature_label, axis=1)

# Display the DataFrames
print("Prompt activations:")
display(prompt_df)

print("\nOutput activations:")
display(output_df)

Prompt activations:


,feature,sum_activation,mean_activation,feature_label
0,5419,13.926304,13.926304,information related to credit unions and financial matters
1,10035,3.486936,3.486936,information related to credit card transactions in an online context
2,471,2.929296,2.929296,terms and phrases related to credit ratings and financial assessments by agencies.
3,3869,2.272645,2.272645,descriptions related to food preparation and ingredients
4,9779,2.013997,2.013997,questions pertaining to specific instructions or guidelines



Output activations:


,feature,sum_activation,mean_activation,feature_label
0,471,362.772522,1.711191,terms and phrases related to credit ratings and financial assessments by agencies.
1,3464,250.467194,1.181449,terms related to financial advice and credit improvement
2,3553,226.805420,1.069837,research papers related to health and medicine
3,5419,166.543854,0.785584,information related to credit unions and financial matters
4,6351,121.404739,0.572664,"references to scoring, typically in the context of sports"


In [121]:
def add_label(prompt_df, output_df, apiKey, modelId, layer):
    """Applies feature labels, styles, and displays the dataframes."""

    def get_feature_label(row):
        feature_index = int(row['feature'])
        description = get_or_generate_activation_description(modelId, layer, feature_index, apiKey)
        return description

    # Apply to output_df
    output_df['feature_label'] = output_df.apply(get_feature_label, axis=1)

    # Apply to prompt_df
    prompt_df['feature_label'] = prompt_df.apply(get_feature_label, axis=1)

    return prompt_df, output_df

## Hallucination detection


In [122]:
prompt = "financial"
apiKey = userdata.get('NEURONPEDIA_API_KEY')
modelId = "gemma-2b"
layer = "6-res-jb"
top_n = 10
prompt_df, output_df, output = get_top_activations(model, sae, prompt, top_n=top_n)
prompt_df_updated, output_df_updated = add_label(prompt_df, output_df, apiKey, modelId, layer)


In [123]:
# Display the DataFrames
pd.set_option('display.max_colwidth', None)
print("Prompt activations:")
display(prompt_df_updated)

# print("\nOutput activations:")
# display(output_df_updated)

#print(output)

Prompt activations:


,feature,sum_activation,mean_activation,feature_label
0,15857,16.145458,16.145458,financial terms and information
1,11912,2.184104,2.184104,"text mentioning technical details and updates, including dates, times, and versions"
2,6513,1.574638,1.574638,"references to education and business institutions, particularly relating to specific programs, schools, and disciplines"
3,3586,1.407619,1.407619,attends from the beginning-of-sequence marker to the first token in the sentence.
4,15173,1.335189,1.335189,terms related to real estate and local government activities
5,544,1.236598,1.236598,"terms related to medical conditions and medications, specifically focusing on cholesterol-lowering drugs"
6,11938,1.011851,1.011851,technical and pharmaceutical information
7,15369,0.971441,0.971441,news articles cited from the Associated Press
8,3869,0.968602,0.968602,descriptions related to food preparation and ingredients
9,7741,0.964319,0.964319,phrases related to personal websites or blogs


In [23]:
# Function to retrieve synthetic RAG data for finance
def get_rag_data(prompt):
    # This is where actual RAG retrieval would occur
    return (
        "According to Bloomberg and SEC filings, the stock market trends indicate volatility. "
        "Macroeconomic factors, inflation, and Federal Reserve policies significantly impact these trends. "
        "Always verify insights from sources such as Reuters, Bloomberg, or official financial statements."
    )

def generate_response(prompt):


    variant.reset()
    variant.abort_when(finance_features[0] > 0.25)  # This will not raise an error but will silently stop if triggered

    generated_tokens = ""

    try:
        # Generate initial response
        response = client.chat.completions.create(
            [
                {"role": "user", "content": prompt}
            ],
            model=variant,
            stream=True,
            max_completion_tokens=500,
        )

        for token in response:
            if token.choices[0].delta.content:
                generated_tokens += token.choices[0].delta.content

        # Print the initial generated response
        print("\n--- GENERATED RESPONSE ---\n")
        print(generated_tokens)

    except Exception as e:
        print(f"\nError: {e} - Falling back to RAG\n")

    # Always fall back to RAG if hallucination threshold was crossed
    rag_data = get_rag_data(prompt)
    print("\n--- NEW TOKENS (RAG-Verified Data) ---\n")

    variant.reset()

    response = client.chat.completions.create(
        [
            {"role": "system", "content": "You are a financial assistant providing fact-based insights. "
                                          "Use only verified data from Bloomberg, Reuters, or SEC filings. "
                                          "Here is the retrieved data: " + rag_data},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": generated_tokens},
        ],
        model=variant,
        stream=True,
        max_completion_tokens=500,
    )

    for token in response:
        if token.choices[0].delta.content:
            print(token.choices[0].delta.content, end="")

    return None

# **Finance Example Query to Check for Hallucination Handling**
generate_response("What are the expected Federal Reserve rate decisions for next quarter?")